# Projet P7 : Implémentez un modèle de scoring

## Contexte

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.)

<b> MISSION 1 : </b>

Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

<b> Approche MLOps : </b>

Afin de pouvoir faire évoluer régulièrement le modèle, mettre en œuvre une démarche de type MLOps d’automatisation et d’industrialisation de la gestion du cycle de vie du modèle (du tracking des expérimentations à l’analyse en production du data drift). 

Mettre en oeuvre au minimum les étapes orientées MLOps suivantes : 

- Dans le notebook d’entraînement des modèles, générer à l’aide de MLFlow un tracking d'expérimentations
- Lancer l’interface web 'UI MLFlow" d'affichage des résultats du tracking
- Réaliser avec MLFlow un stockage centralisé des modèles dans un “model registry”
- Tester le serving MLFlow
- Gérer le code avec le logiciel de version Git
- Partager le code sur Github pour assurer une intégration continue
- Utiliser Github Actions pour le déploiement continu et automatisé du code de l’API sur le cloud
- Concevoir des tests unitaires avec Pytest (ou Unittest) et les exécuter de manière automatisée lors du build réalisé par Github Actions

<b> Elaboration du modèle : </b>

Attention à deux points spécifiques au contexte métier : 

- Le déséquilibre entre le nombre de bons et de moins bons clients doit être pris en compte pour élaborer un modèle pertinent, avec une méthode au choix
- Le déséquilibre du coût métier entre un faux négatif (FN - mauvais client prédit bon client : donc crédit accordé et perte en capital) et un faux positif (FP - bon client prédit mauvais : donc refus crédit et manque à gagner en marge). Vous pourrez supposer, par exemple, que le coût d’un FN est dix fois supérieur au coût d’un FP. Vous créerez un score “métier” (minimisation du coût d’erreur de prédiction des FN et FP) pour comparer les modèles, afin de choisir le meilleur modèle et ses meilleurs hyperparamètres. Attention cette minimisation du coût métier doit passer par l’optimisation du seuil qui détermine, à partir d’une probabilité, la classe 0 ou 1 (un “predict” suppose un seuil à 0.5 qui n’est pas forcément l’optimum). En parallèle, maintenez pour comparaison et contrôle des mesures plus techniques, telles que l’AUC et l’accuracy.

## Etapes du projet :

Elaboration d'un modèle de prédiction sous forme d’une API qui permet de calculer la probabilité de défaut du client, ainsi que sa classe (accepté ou refusé), déployer l'API sur une plateforme Cloud.

- <b> Etape préliminaire : </b> Importation des données ( --> NoteBook1 : Preparation des données )
- <b> Etape 1 : </b> EDA et feature engineering sur la table principale application ( --> NoteBook1 ) 
- <b> Etape 2 : </b> Ajout des tables bureau et bureau_balance ( --> NoteBook1 )
- <b> Etape 3 : </b> Ajout des tables previous_application, POS_CASH_balance, installments_payments et credit_card_balance ( --> NoteBook1 )

- <b> Etape 4 : </b> EDA et Feature selection, feature engineering ( --> NoteBook1 )

- <b> Etape 5 : </b> Elaboration des modèles avec un tracking d'expérimentations (avec Cross-Validation et optimisation des hyperparamètres, via GridsearchCV ou équivalent)
- <b> Etape 6 : </b> Création de l'API (Notebook ou une application Streamlit pour réaliser en local l’interface de test de l’API)
- <b> Etape 7 : </b> Déploiement de l’API sur une plateforme Cloud (de préférence une solution gratuite)
- <b> Etape 8 : </b> Analyse du Data Drift (evidently)

## Imports

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn methods
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from flask import Flask, request, jsonify
import joblib

# Outils MLOps
import mlflow
import mlflow.sklearn

## ETAPE 6 : Création de l'API

ETAPE 6 : Création d'un fichier .py séparé pour notre application Flask = ce fichier importera le modèle et définira les endpoints pour prédire les résultats.
    
    1. Importer les données clients à prédire 
    2. Transformer les données 
        - standardisation 
        - imputation
    3. Application Flask - fichier python
    4. Intéroger l'API

### 1. Importer les données clients à prédire 

In [5]:
data_client = pd.read_csv('./data/test_small.csv')

In [6]:
data_client = data_client.drop(columns = ['SK_ID_CURR'])

In [7]:
data_client.shape

(48744, 276)

### 2. Transformer les données 

#### -- Standardiser --

In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_client)

#### -- Imputer les valeurs manquantes --

In [15]:
# Créer une instance de SimpleImputer avec la stratégie 'median'
imputer = SimpleImputer(strategy='median')

# Appliquer l'imputer sur X_scaled
X_scaled_imputed = imputer.fit_transform(X_scaled)

### 3. Création de l'API

Cette partie n'est pas exécutée ici mais dans un script app.py

In [70]:
def cout_metier(y_true, y_pred):
    """Cette fonction calcule le coût métier à partir de la matrice de confusion : 10*FN + FP."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return 10 * fn + fp

cout_metier_scorer = make_scorer(cout_metier, greater_is_better=False)

def find_best_threshold(estimator, X, y):
    """Cette fonction trouve le seuil optimal en testant une gamme de seuils et en choisissant celui avec le score métier le plus bas."""
    thresholds = np.linspace(0, 1, 101)
    best_threshold, best_score = 0, float('inf')
    for threshold in thresholds:
        y_pred = (estimator.predict_proba(X)[:, 1] >= threshold).astype(int)
        score = cout_metier(y, y_pred)
        if score < best_score:
            best_threshold, best_score = threshold, score
    return best_threshold, best_score

In [71]:
# Charger le modèle
model = joblib.load('modele_logRegression.pkl')

In [75]:
# Récupérer le meilleur modèle et la partie classification
best_model = model.best_estimator_
logistic_model = best_model.named_steps['classification']

In [ ]:
# Initialiser l'application Flask
app = Flask(__name__)

# Définir un endpoint pour la prédiction
@app.route('/predict', methods=['POST'])
def predict():
    # Récupérer les données envoyées dans la requête POST
    data = request.get_json(force=True)
    
    # Extraire les caractéristiques et les convertir en tableau NumPy
    features = np.array(data).reshape(1, -1)
    
    # Faire une prédiction
    prediction = logistic_model.predict(features)
    
    # Vérifier le résultat de la prédiction et retourner le message approprié
    result = "crédit accepté" if int(prediction[0]) == 0 else "crédit refusé"
    
    # Retourner le résultat sous forme de JSON
    return jsonify({'prediction': result})

# Définir un endpoint de test pour s'assurer que l'API fonctionne
@app.route('/', methods=['GET'])
def home():
    return "API de prédiction de régression logistique est opérationnelle!"

# Lancer l'application Flask
if __name__ == '__main__':
    app.run(debug=True)

### 4. Intéroger l'API

In [54]:
# Selectionner un client :
client = X_scaled_imputed [1,:]

In [56]:
import requests

url = 'http://127.0.0.1:5000/predict'
data = client.tolist()

response = requests.post(url, json=data)
print(response.json())

{'prediction': 'crédit refusé'}
